In [3]:
import os
import re
from fnmatch import fnmatch
from collections import defaultdict

# Define a dictionary to store API invocations and their response details
api_invocations = defaultdict(list)

# Specify the directory to search
search_directory = "C:\\Users\\inbra\\OneDrive - Software AG\\Documents\\Adda\\\go-live\\07-09-2023"

# Function to process log files
def process_log_file(file_path):
    with open(file_path, 'r') as f:
        log_lines = f.readlines()

    for line in log_lines:
        api_info_match = re.search(r'\[APIINFO: (.*?)\]', line)
        response_code_match = re.search(r'\[ProviderResponseCode=(\d+)\|', line)
        response_time_match = re.search(r'\[RESPONSETIME: (\d+) ms\]', line)

        if api_info_match and response_code_match and response_time_match:
            api_info = api_info_match.group(1)
            response_code = int(response_code_match.group(1))
            response_time = int(response_time_match.group(1))

            if response_code == 200:
                result = "success"
            else:
                result = "failure"

            if response_time > 1000:
                response_time_status = "high_response_time"
            else:
                response_time_status = "normal_response_time"

            api_invocations[api_info].append((result, response_time_status))

# Recursively search for log files in the directory and its subdirectories
for root, dirs, files in os.walk(search_directory):
    for file in files:
        if fnmatch(file, "*.log"):
            file_path = os.path.join(root, file)
            process_log_file(file_path)

# Print results
for api_info, invocations in api_invocations.items():
    total_invocations = len(invocations)
    success_count = sum(1 for result, _ in invocations if result == "success")
    failure_count = total_invocations - success_count
    high_response_time_count = sum(1 for _, response_time_status in invocations if response_time_status == "high_response_time")

    print(f"API: {api_info}")
    print(f"Total Invocations: {total_invocations}")
    print(f"Success: {success_count} | Failure: {failure_count}")
    print(f"High Response Time: {high_response_time_count}")
    print("-" * 30)


API: ADCustomsGenericServices_1.0
Total Invocations: 53096
Success: 52964 | Failure: 132
High Response Time: 12073
------------------------------
